In [6]:
import numpy as np
from scipy.io import loadmat
import os

## radar to lidar transform (move radar to lidar) <=> lidar to radar porjection matrix

In [ ]:
tr_rl_translation = np.array([-2.54, 0.3, 0.7]) # x, y, z (m)

# bbox_zyx_radar =  bbox_zyx_lidar + tr_rl_translation
# how to represent bbox in radar coordinate? add tr_rl_translation to the bbox center

In [10]:
# physical valuses of each DEAR tensors' cell
def load_physical_values():
    temp_values = loadmat(os.path.join('/mnt/ssd1/kradar_dataset', 'resources', 'info_arr.mat'))
    arr_range = temp_values['arrRange']
    deg2rad = np.pi/180.
    arr_azimuth = temp_values['arrAzimuth']*deg2rad
    arr_elevation = temp_values['arrElevation']*deg2rad
    arr_range = arr_range.flatten()
    arr_azimuth = arr_azimuth.flatten()
    arr_elevation = arr_elevation.flatten()
    arr_doppler = loadmat(os.path.join('/mnt/ssd1/kradar_dataset', 'resources', 'arr_doppler.mat'))['arr_doppler']
    arr_doppler = arr_doppler.flatten()
    return arr_range, arr_azimuth, arr_elevation, arr_doppler

In [13]:
arr_range, arr_azimuth, arr_elevation, arr_doppler = load_physical_values()
rad2deg = 180./np.pi

In [17]:
print(arr_range[0], arr_range[-1])
print(arr_azimuth[0]*rad2deg, arr_azimuth[-1]*rad2deg)
print(arr_elevation[0]*rad2deg, arr_elevation[-1]*rad2deg)

print(arr_azimuth.shape)
print(arr_elevation.shape)
print(arr_range.shape)

print(arr_range[:2])

0.0 118.037109375
-53.0 53.0
-18.0 18.0
(107,)
(37,)
(256,)
[0.         0.46289062]


In [19]:
radar_ear = np.flip(np.load('/mnt/ssd1/kradar_dataset/radar_tensor/11/radar_DEAR_D_downsampled_2/tesseract_00300.npy')).mean(axis=0)


In [20]:
radar_ear.shape

(37, 107, 256)

In [23]:
# if cfar output is 1 at matrix index (e, a, r) = (0, 0, 50)
coord_ear = (arr_elevation[0], arr_azimuth[0], arr_range[50])

In [24]:
coord_ear

(-0.3141592653589793, -0.9250245035569946, 23.14453125)

In [25]:
def polar_to_cart(ear_coord):
    # TODO: return a list of points in cartesian coordinate if cfar_output of that cell is 1
    e, a, r = ear_coord
    x = r * np.cos(a) * np.cos(e)
    y = r * np.sin(a) * np.cos(e)
    z = r * np.sin(e)
    return x, y, z



In [26]:
x, y, z = polar_to_cart(coord_ear)

In [27]:
print(x, y, z)

13.247006206193223 -17.57937098790215 -7.152053483092044


In [ ]:
def apply_cfar(matrix, num_guard_cells, num_training_cells, threshold_factor):
    # Get matrix dimensions
    elevation_dim, azimuth_dim, range_dim = matrix.shape
    
    # Initialize an empty matrix to store CFAR output
    cfar_output = np.zeros_like(matrix)
    
    # Loop through each cell in the 4D matrix
    for r in range(range_dim):
        for e in range(elevation_dim):
            for a in range(azimuth_dim):
                # Define the grid for training cells
                training_cells = []
                for tr in range(r-num_guard_cells-num_training_cells, r+num_guard_cells+num_training_cells+1):
                    for te in range(e-num_guard_cells-num_training_cells, e+num_guard_cells+num_training_cells+1):
                        for ta in range(a-num_guard_cells-num_training_cells, a+num_guard_cells+num_training_cells+1):
                            if (abs(tr-r)>num_guard_cells and
                                abs(te-e)>num_guard_cells and abs(ta-a)>num_guard_cells):
                                if 0 <= tr < range_dim and 0 <= te < elevation_dim and 0 <= ta < azimuth_dim:
                                    training_cells.append(matrix[te, ta, tr])
                
                # Calculate noise threshold based on training cells
                threshold = np.mean(training_cells) * threshold_factor
                # Test against the threshold
                if matrix[te, ta, tr] > threshold:
                    cfar_output[e, a, r] = 1
    return cfar_output


# Apply 4D CFAR detection
num_guard_cells = 1
num_training_cells = 2
threshold_factor = 1.2


cfar_output = apply_cfar(radar_ear, num_guard_cells, num_training_cells, threshold_factor)
# cfar_output will contain ones at positions where peaks are detected and zeros elsewhere


cfar_points = polar_to_cart(cfar_output)


## TODO
1. Test CFAR on one frame
    - Read DEAR, take avg along doppler axis
    - do CFAR on DEAR
    - for each cell =1, save its corresponding (x,y,z)
    - save the xyz in pcd format (format follows os2-64)
    - visualize pcd on labeling tool

2.  Test radar visibility on one frame (be careful about the coordinate transform!!!)
    - load the pcd file
    - for each bbox, check if any CRAR point falls inside
    - add radar visibility attribute to the label json file

3. Test on one sequence

4. Run on all sequences

    
